In [ ]:
import os
import json
import cv2
from glob import glob
import numpy as np

image_dir = "../SandBox_Area/yolov11_Propane/test/images"  
label_dir = "../SandBox_Area/yolov11_Propane/test/labels"  
output_json = "coco_annotations.json"

class_names = ['Garage', 'House', 'Other', 'Propane', 'Trailer']
category_mapping = {i: name for i, name in enumerate(class_names)}

coco_output = {
    "images": [],
    "annotations": [],
    "categories": [{"id": i, "name": name, "supercategory": "none"} for i, name in enumerate(class_names)]
}

annotation_id = 1
image_id = 1

def yolo_to_coco_segmentation(segmentation, img_width, img_height):
    points = []
    for i in range(0, len(segmentation), 2):
        x = float(segmentation[i]) * img_width
        y = float(segmentation[i + 1]) * img_height
        points.extend([x, y])
    return [points] 

for img_path in glob(os.path.join(image_dir, "*.jpg")) + glob(os.path.join(image_dir, "*.png")):
    filename = os.path.basename(img_path)
    image = cv2.imread(img_path)
    height, width = image.shape[:2]

    coco_output["images"].append({
        "id": image_id,
        "file_name": filename,
        "width": width,
        "height": height
    })

    label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
    if not os.path.exists(label_path):
        image_id += 1
        continue

    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 8:
                continue  # Skip incomplete annotations
            class_id = int(parts[0])
            segmentation = parts[5:]  
            seg_points = yolo_to_coco_segmentation(segmentation, width, height)

            coco_output["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "segmentation": seg_points,
                "area": cv2.contourArea(
                    cv2.UMat(
                        np.array(seg_points[0], dtype=np.float32).reshape(-1, 2)
                    )
                ),
                "bbox": cv2.boundingRect(
                    np.array(seg_points[0], dtype=np.float32).reshape(-1, 2)
                ),
                "iscrowd": 0
            })
            annotation_id += 1

    image_id += 1

with open(output_json, "w") as f:
    json.dump(coco_output, f, indent=2)

print(f"COCO annotation saved to {output_json}")


COCO annotation saved to coco_annotations.json
